#  Setup stuff: Connectivity

In [1]:

#  Setting display options

from tabulate import tabulate
import pandas as pd
   #
pd.set_option("display.width", 480)

print("--")


--


In [2]:

from katana import remote
from katana.remote import import_data


my_client = remote.Client()

print(my_client)


In [3]:

NUM_PARTITIONS  = 3
   #
DB_NAME         = "my_db"
GRAPH_NAME      = "my_graph"

print("--")


--


In [4]:
#  DELETE ALL GRAPHS

for l_database in my_client.databases():
   for l_graph in my_client.get_database(name=l_database.name).graphs_in_database():
      l_handle=my_client.get_database(name=l_database.name).get_graph_by_id(id=l_graph.graph_id)
      l_handle.delete()

for l_graph in my_client.graphs():
   print("GRAPH ID: ", l_graph.graph_id, "      GRAPH Version: ", l_graph.version)

print("--")


--


In [ ]:
#  DELETE ALL DATABASES

for l_database in my_client.databases():
   if (l_database.name != "default"):
      my_client.get_database(name=l_database.name).delete_database()
      print("--")

for l_database in my_client.databases():
   print("DB ID: ", l_database.database_id, "     DB Name: ", l_database.name)


In [ ]:
#  CREATE DATABASE

my_database = my_client.create_database(name=DB_NAME)

print(my_database.database_id)


In [5]:
#  CREATE GRAPH
#
my_graph = my_client.get_database(name=DB_NAME).create_graph(name=GRAPH_NAME, num_partitions=NUM_PARTITIONS)

print(my_graph)


<_Graph my_graph, pHG8gqFofZmrzwdNd5q2q8LvpN7Ptf7BDsx62K7hWnT, 0>


# Create: Vertices/nodes, edges ..

In [6]:

#  Vertices/Nodes, Airports ..

df_airports = pd.DataFrame([
       #
   ["MKE", "Milwaukee"               , "Airport"],
   ["ORD", "Chicago O-Hare"          , "Airport"],
   ["SJC", "San Jose"                , "Airport"],
   ["DEN", "Denver"                  , "Airport"],
       #
   ], columns = ["airport_code", "airport_name", "LABEL"])

print(tabulate(df_airports, headers='keys', tablefmt='psql'))


+----+----------------+----------------+---------+
|    | airport_code   | airport_name   | LABEL   |
|----+----------------+----------------+---------|
|  0 | MKE            | Milwaukee      | Airport |
|  1 | ORD            | Chicago O-Hare | Airport |
|  2 | SJC            | San Jose       | Airport |
|  3 | DEN            | Denver         | Airport |
+----+----------------+----------------+---------+


In [7]:
#  Create the edge, flights ..

df_flights = pd.DataFrame([
      #
   ["MKE", "ORD",   66, 1, "FLIES_TO" ],
   ["ORD", "MKE",   66, 1, "FLIES_TO" ],
      #
   ["ORD", "DEN",  886, 1, "FLIES_TO" ],
   ["DEN", "ORD",  886, 1, "FLIES_TO" ],
      #
   ["SJC", "DEN",  948, 1, "FLIES_TO" ],                           #  Notice SJC flies to/from Denver
   ["DEN", "SJC",  948, 1, "FLIES_TO" ],
      #
   ["SJC", "ORD", 1829, 1, "FLIES_TO" ],                           #  Notice SJC flies to ORD, but not ORD to SJC
      #
   ], columns = ["START_ID", "END_ID", "DISTANCE", "NUM_HOPS", "TYPE"])

print(tabulate(df_flights, headers='keys', tablefmt='psql'))


+----+------------+----------+------------+------------+----------+
|    | START_ID   | END_ID   |   DISTANCE |   NUM_HOPS | TYPE     |
|----+------------+----------+------------+------------+----------|
|  0 | MKE        | ORD      |         66 |          1 | FLIES_TO |
|  1 | ORD        | MKE      |         66 |          1 | FLIES_TO |
|  2 | ORD        | DEN      |        886 |          1 | FLIES_TO |
|  3 | DEN        | ORD      |        886 |          1 | FLIES_TO |
|  4 | SJC        | DEN      |        948 |          1 | FLIES_TO |
|  5 | DEN        | SJC      |        948 |          1 | FLIES_TO |
|  6 | SJC        | ORD      |       1829 |          1 | FLIES_TO |
+----+------------+----------+------------+------------+----------+


In [8]:

# Import the 2 previously created Python DataFrames into KatanaGraph ..

with import_data.DataFrameImporter(my_graph) as df_importer:   
    
   df_importer.nodes_dataframe(df_airports,                    #  Aiports set of Nodes
      id_column             = "airport_code",
      id_space              = "Airport",  
      label                 = "Airport",  
      )
   
   df_importer.edges_dataframe(df_flights,                     #  Our Edge, specifying the relationship between Airport --> FLIES_TO --> Airport
      source_id_space       = "Airport", 
      destination_id_space  = "Airport",   
      source_column         = "START_ID",
      destination_column    = "END_ID",
      type                  = "FLIES_TO"
      )

   df_importer.insert()

print("--")


          0/? [?op/s]

          0/? [?op/s]

--


In [ ]:

#  What's in the graph ..

l_result1 = my_graph.query("""
   MATCH ( n ) 
   RETURN n
   """)
# print(l_result1[0:30])
print(tabulate(l_result1, headers='keys', tablefmt='psql'))

l_result2 = my_graph.query("""
   MATCH ( n ) - [ r ] -> ( m )
   RETURN r
   """)
# print(l_result2[0:30])
print(tabulate(l_result2, headers='keys', tablefmt='psql'))



#  Actual bfs() ..

In [10]:

# import numpy as np
from katana import remote
from katana.remote import analytics
# from katana_enterprise.distributed import PartitionPolicy

#  Used to generate a unique output column name
#
l_cntr = 20

print("--")


--


In [19]:

#  Running analytics.bfs() ..


l_source_nodeid       = "2"                        #  id of node to start on (this is SJC)



#  Generate a unique prop name so we can run this over and over without edit ..
#
# l_cntr += 1
   #
l_output_propname     = "bfs" + str(l_cntr).zfill(4)
l_max_iterations      = 10000 


analytics.bfs(
   my_graph,
   source                  = l_source_nodeid,
   result_property_name    = l_output_propname,
   max_iterations          = l_max_iterations,
   #  delta                   = 5
   )


display(print("--"))


          0/? [?op/s]

OperationError: E9UEYj4d5DjkrUgo1UiuMLvCKsVh3z3ChK9d57NfjJNj-4EE59uUYb2cJHV5Md backtrace (AnalyticsOperation.cpp:1132): backtrace (AnalyticsOperation.cpp:903): backtrace (AnalyticsOperation.cpp:263): backtrace (Bfs.cpp:280): property with name bfs0024 already exists (PropertyTableGroup.cpp:493): invalid argument: invalid argument

In [18]:

#  My notes,
#
#     .  If "source" id is not found, no value is output. Yet, routine complains with,
#           >>> OperationError: E9UEYj4d5DjkrUgo1UiuMLvCKsVh3z3ChK9d57NfjJNj-4EE59uUYb2cJHV5Md backtrace
#           >>> (AnalyticsOperation.cpp:1132): backtrace (AnalyticsOperation.cpp:903): backtrace (AnalyticsOperation.cpp:263): backtrace (Bfs.cpp:280): property with name bfs0024 already exists (PropertyTableGroup.cpp:493): invalid argument: invalid argument




l_result1 = my_graph.query(
   """
   MATCH (n) 
   RETURN n
   """ )
print(tabulate(l_result1, headers='keys', tablefmt='psql'))

l_result2 = my_graph.query(
   """
   MATCH (n) - [r] -> (m)
   RETURN r
   """ )
print(tabulate(l_result2, headers='keys', tablefmt='psql'))


          0/? [?op/s]

+----+-----------------+-------------+-----------+------------------+------------------+----------+
|    |   n.internal_id | n.labels    | n.LABEL   | n.airport_code   | n.airport_name   | n.type   |
|----+-----------------+-------------+-----------+------------------+------------------+----------|
|  0 |               0 | ['Airport'] | Airport   | DEN              | Denver           | node     |
|  1 | 281474976710656 | ['Airport'] | Airport   | MKE              | Milwaukee        | node     |
|  2 | 562949953421313 | ['Airport'] | Airport   | SJC              | San Jose         | node     |
|  3 | 562949953421312 | ['Airport'] | Airport   | ORD              | Chicago O-Hare   | node     |
+----+-----------------+-------------+-----------+------------------+------------------+----------+


          0/? [?op/s]

+----+-----------------+----------------+-----------------+-----------+--------------+--------------+----------+-----------------+------------------+----------+
|    |        r.end.id | r.end.labels   |   r.internal_id | r.label   |   r.DISTANCE |   r.NUM_HOPS | r.TYPE   |      r.start.id | r.start.labels   | r.type   |
|----+-----------------+----------------+-----------------+-----------+--------------+--------------+----------+-----------------+------------------+----------|
|  0 | 562949953421313 | ['Airport']    |               0 | FLIES_TO  |          948 |            1 | FLIES_TO |               0 | ['Airport']      | edge     |
|  1 | 562949953421312 | ['Airport']    |               1 | FLIES_TO  |          886 |            1 | FLIES_TO |               0 | ['Airport']      | edge     |
|  2 | 562949953421312 | ['Airport']    | 281474976710656 | FLIES_TO  |           66 |            1 | FLIES_TO | 281474976710656 | ['Airport']      | edge     |
|  3 | 562949953421312 | ['Airport